**Learning goals:**

> - Load CSV and JSON files/
> - Inspect, clean, and document raw tables.  
> - Convert *wide* data to *long* form and back.  
> - Build concise, readable **pandas pipelines** via method chaining.


In [12]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style="whitegrid")
DATA_DIR = Path('../../') / 'data'   # adjust if notebook is elsewhere


### 1  Loading CSV Files

We’ll use a small **Yelp reviews** sample (CSV) to illustrate common options.

In [7]:
csv_path = DATA_DIR / 'yelp.csv'
yelp = pd.read_csv(csv_path)
yelp.head()
# Source of data: https://www.kaggle.com/datasets/omkarsabnis/yelp-reviews-dataset

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


*Always* peek at shape, dtypes, and memory footprint:

In [ ]:
print("Rows:", len(yelp))
print(yelp.dtypes)
print(yelp.memory_usage(deep=True).sum() / 1_048_576, "MB")

Rows: 10000
business_id    object
date           object
review_id      object
stars           int64
text           object
type           object
user_id        object
cool            int64
useful          int64
funny           int64
dtype: object
11.12995433807373 MB


#### 1.1  Parsing dates on read
If a column holds ISO date strings, parse immediately to save trouble later:

In [8]:
yelp['date'].head()

0    2011-01-26
1    2011-07-27
2    2012-06-14
3    2010-05-27
4    2012-01-05
Name: date, dtype: object

In [9]:
yelp = pd.read_csv(csv_path, parse_dates=['date'])
yelp['date'].head()

0   2011-01-26
1   2011-07-27
2   2012-06-14
3   2010-05-27
4   2012-01-05
Name: date, dtype: datetime64[ns]

#### 1.2  Missing‑value flags
CSV sometimes uses `-999`, `N/A`, or blank strings to mean *missing*. We can examine the spotify data as an example. 

In [10]:

ratings_path = DATA_DIR / 'spotify_track_ratings.csv'
ratings = pd.read_csv(ratings_path, na_values=['-999', 'n/a', ''])
ratings.isna().mean().sort_values(ascending=False).head()
# source of data: https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset/data

artists       0.000009
album_name    0.000009
track_name    0.000009
Unnamed: 0    0.000000
mode          0.000000
dtype: float64

### 2  Loading JSON

The `citibike station information` exports logs in JSON. We can read and turn it into a data frame with `json_normalize` in the `json` library.

In [14]:
import json

json_path = DATA_DIR / 'citibike_station_information.json'

with open(json_path) as f:
    raw = json.load(f)

stations = pd.json_normalize(raw["data"]["stations"])
stations.head()

# source of data: https://gbfs.citibikenyc.com/gbfs/en/station_information.json

,external_id,region_id,rental_methods,short_name,station_type,has_kiosk,name,station_id,eightd_station_services,capacity,electric_bike_surcharge_waiver,eightd_has_key_dispenser,lon,lat,rental_uris.ios,rental_uris.android
0,66dc2c78-0aca-11e7-82f6-3863bb44ef7c,71,"[KEY, CREDITCARD]",6779.05,classic,True,Broadway & W 53 St,66dc2c78-0aca-11e7-82f6-3863bb44ef7c,[],85,False,False,-73.982681,40.763441,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
1,0b009276-767c-47ee-a4d2-7633d4f95d95,71,"[KEY, CREDITCARD]",6667.04,classic,True,W 47 St & 6 Ave,0b009276-767c-47ee-a4d2-7633d4f95d95,[],62,False,False,-73.982550,40.758397,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
2,66dbc420-0aca-11e7-82f6-3863bb44ef7c,71,"[KEY, CREDITCARD]",5303.06,classic,True,Clinton St & Grand St,66dbc420-0aca-11e7-82f6-3863bb44ef7c,[],51,False,False,-73.987030,40.715595,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
3,ffae66ec-7c16-436f-bd0a-eedf81d580e7,71,"[KEY, CREDITCARD]",8778.01,classic,True,E Mosholu Pkwy & Van Cortlandt Ave E,ffae66ec-7c16-436f-bd0a-eedf81d580e7,[],22,False,False,-73.883660,40.876560,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan
4,2637fed8-1e2a-460b-83eb-3eb32ace0f4e,71,"[KEY, CREDITCARD]",6398.08,classic,True,E 35 St & Madison Ave,2637fed8-1e2a-460b-83eb-3eb32ace0f4e,[],50,False,False,-73.982556,40.748480,https://bkn.lft.to/lastmile_qr_scan,https://bkn.lft.to/lastmile_qr_scan


### 3  Wide and Long Formats

- A long format has one observation per row, one variable per column. Repeated measurements or values for the same subject appear in multiple rows. This is also known as the *tidy* format.

- A **wide format** keeps repeated measurements or values for the same subject spread across multiple columns, rather than multiple rows.

#### 3.1  Example: Spotify audio features across years (wide)

In [17]:

wide = pd.DataFrame({
    'feature': ['danceability', 'energy', 'acousticness'],
    '2018': [0.62, 0.55, 0.18],
    '2019': [0.63, 0.57, 0.16],
    '2020': [0.61, 0.53, 0.19],
})
wide


,feature,2018,2019,2020
0,danceability,0.62,0.63,0.61
1,energy,0.55,0.57,0.53
2,acousticness,0.18,0.16,0.19


Transform to the long format via `melt()` 

In [18]:

tidy = (wide
        .melt(id_vars='feature', var_name='year', value_name='value')
        .assign(year=lambda d: d['year'].astype(int)))
tidy.head()


,feature,year,value
0,danceability,2018,0.62
1,energy,2018,0.55
2,acousticness,2018,0.18
3,danceability,2019,0.63
4,energy,2019,0.57


Restore to the wide format via `pivot()` (or `pivot_table`)

In [20]:
wide_again = tidy.pivot(index='feature', columns='year', values='value')
wide_again

year,2018,2019,2020
feature,,,
acousticness,0.18,0.16,0.19
danceability,0.62,0.63,0.61
energy,0.55,0.57,0.53
